In [17]:
import pandas as pd
import shapely.geometry as geom
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import mapping
from shapely.geometry import Polygon
import rasterio
from rasterio.mask import mask
import numpy as np
from PIL import Image
import os
from affine import Affine
%matplotlib inline

In [18]:
pics = gpd.read_file("pictures/pictures.shp")
fields = gpd.read_file("Crops_2016/Crops_2016_26910.shp")

In [19]:
pic1= pics.iloc[0]["geometry"]
def container(x):
    """This function applies the contain function which 
    determins if the coordinates of crop field are in a photo
    """
    return(pic1.intersects(x))
intersect_fields = fields[fields['geometry'].apply(container)]

In [39]:
intersect_fields

OBJECTID  county_num  pk_site_id site_id  pk_permit_ permit_num  \
368     2965866          57     7052472    MEB1      821762    0600132   
369     2966432          57     7052473    MEB2      821762    0600132   
370     2965867          57     7052474    MEB3      821762    0600132   
371     2966644          57     7052475    MEB4      821762    0600132   
372     2967427          57     7052476    MEB5      821762    0600132   
575     3037524          57     7066366     JM2      822649    0600033   
598     3039338          57     7066390  ER55C2      822649    0600033   
757     3074879          57     7078375     A69      823620    570367A   
760     3117074          57     7078384    G220      823620    570367A   
811     3107816          57     7082228   A217W      823859    0600712   
812     3107571          57     7082229   A218E      823859    0600712   
815     3092233          57     7082232    C56J      823859    0600712   
816     3093485          57     7082233    C56K      823859    0600712   
817     3093486          57     7082234    C56L      823859    0600712   
1159    3204785          57     7106576    MEB1      825537    0600132   
1160    3204297          57     7106577    MEB2      825537    0600132   
1161    3204298          57     7106578    MEB3      825537    0600132   
1162    3203523          57     7106579    MEB4      825537    0600132   
1163    3205951          57     7106580    MEB5      825537    0600132   
1168    3204304          57     7106645    MEB1      825544    0600132   
1169    3203529          57     7106646    MEB2      825544    0600132   
1170    3226497          57     7106647    MEB3      825544    0600132   
1171    3203989          57     7106648    MEB4      825544    0600132   
1172    3226498          57     7106649    MEB5      825544    0600132   
1232    3226629          57     7107626     JM2      825634    0600033   
1255    3217429          57     7107649  ER55C2      825634    0600033   
1273    3219003          57     7108203   A217W      825668    0600712   
1274    3219004          57     7108204   A218E      825668    0600712   
1277    3219921          57     7108207    C56J      825668    0600712   
1278    3221795          57     7108208    C56K      825668    0600712   
...         ...         ...         ...     ...         ...        ...   
10939   3824896          57     7550534    MEB2      861997    0600132   
10940   3783097          57     7550535    MEB3      861997    0600132   
10941   3800822          57     7550536    MEB4      861997    0600132   
10942   3787400          57     7550537    MEB5      861997    0600132   
10951   3801936          57     7552524     A59      862119    0600189   
10952   3802433          57     7552525     A62      862119    0600189   
10969   3801938          57     7552542     A58      862119    0600189   
11454   3649555          57     7580552     JM2      863375    0600033   
11477   3649556          57     7580575  ER55C2      863375    0600033   
11928   3764102          57     7609808     A59      864629    0600189   
11929   3789105          57     7609809     A62      864629    0600189   
11946   3789107          57     7609826     A58      864629    0600189   
12651   3980753          57     7659761     A18      866551    570101A   
12669   3994754          57     7659780     A19      866551    570101A   
12693   3994758          57     7659811     A29      866551    570101A   
12929   4035660          57     7667291     JM2      866858    0600033   
12952   4037086          57     7667314  ER55C2      866858    0600033   
14023   4041219          57     7765925  JRGF19      869940    570008A   
14209   4092493          57     7770319  JRGF19      870086    570008A   
14800   4247687          57     7821668      K1      873279    570322A   
14801   4210877          57     7821684      K2      873279    570322A   
14802   4247690          57     7821688      K3      873279    570322A   
14803   423

In [9]:
geoms = intersect_fields.geometry.values
field1 =geoms[0]
json_geoms = [mapping(field1)]

In [5]:
#Crops image
with rasterio.open("D:\\NAIP GEOTIFF\\m_3812101_se_10_h_20160711\\m_3812101_se_10_h_20160711.tif", 'r') as src:
    out_image, out_transform = mask(src, json_geoms, crop = True)

In [14]:
out_image.shape[1]*out_image.shape[2]*0.36

634780.7999999999

In [11]:
geoms[0].area

622706.401555776

In [45]:
x = np.swapaxes(out_image.data.transpose(),0,1)
img = Image.fromarray(x, 'RGBA')

In [46]:
src = rasterio.open("D:\\NAIP GEOTIFF\\m_3812101_se_10_h_20160711\\m_3812101_se_10_h_20160711.tif", 'r')

In [47]:
a = src.profile

In [50]:
geoms[0].bounds

(594157.8389296099, 4305840.159636106, 594994.3100869744, 4306597.819774477)

In [51]:
src.affine[0]

0.6

In [52]:
aff = Affine(src.affine[0], src.affine[1], geoms[0].bounds[0], src.affine[3], src.affine[4], geoms[0].bounds[3])
trans = (geoms[0].bounds[0], src.transform[1], src.transform[2],geoms[0].bounds[3], src.transform[4], src.transform[5] )

C:\Users\marce\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
a.update({'height': out_image.shape[1], 'width': out_image.shape[2], 'crs':'epsg:26910', 'affine':aff, 'transform':trans })

In [55]:
b = rasterio.open('p.tif', 'w', **a)

C:\Users\marce\Anaconda3\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [56]:
b.write(out_image)

In [57]:
b.close()

In [58]:
x = rasterio.open('p.tif')

In [59]:
x.bounds

BoundingBox(left=594157.8389296099, bottom=4305839.419774476, right=594994.8389296099, top=4306597.819774477)

In [60]:
x.close()

In [5]:
def plot_field(num = 0):
    path = 'D:\\NAIP GEOTIFF'
    pic_path = os.path.join(os.path.join(path, pics.iloc[num]['name'].replace('.tif', '')),pics.iloc[num]['name'] )
    def container(x):
        return(pics.iloc[num]['geometry'].intersects(x))
    intersect_fields = fields[fields['geometry'].apply(container)]
    fig, ax = plt.subplots(1, figsize=(7,7))
    plt.imshow(plt.imread(pic_path), extent= (pics.iloc[num]['geometry'].bounds[0],pics.iloc[num]['geometry'].bounds[2],pics.iloc[num]['geometry'].bounds[1],pics.iloc[num]['geometry'].bounds[3] ))
    intersect_fields.plot(ax = ax, column = 'crop_list',cmap = 'tab20',legend = True)
    l=ax.get_legend()
    l.set_bbox_to_anchor((1.5, 1, .9, 0))
    ax.set_title("Fields in One Picture by Crop", fontsize = 20)
plot_field(12)

C:\Users\marce\Anaconda3\lib\site-packages\PIL\Image.py:2514: DecompressionBombWarning: Image size (118146000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [ ]:
#This Function will crop all pictures and make files
path = 'D:/NAIP GEOTIFF'
for pic in pics:
    pic_path = os.path.join(path,pic['name'] )
    def container(x):
        return(pic['geometry'].intersects(x))
    intersect_fields = fields[fields['geometry'].apply(container)]
    for i in range(len(intersect_fields)):
        new_geoms = [mapping(intersect_fields.iloc[i].geometry)]
        with rasterio.open(pic_path) as src:
            out_image, out_transform = mask(src, new_geoms, crop = True)
            if(out_image.shape[1]*out_image.shape[2]*0.6**2 >= .5*intersect_fields.iloc[i]["geometry"].area):
                a = src.profile
                aff = Affine(src.affine[0], src.affine[1], intersect_fields.iloc[i]['geometry'].bounds[0], src.affine[3], src.affine[4], geoms[0].bounds[3])
                trans = (intersect_fields.iloc[i]['geometry'].bounds[0], src.transform[1], src.transform[2],intersect_fields.iloc[i]['geometry'].bounds[3], src.transform[4], src.transform[5] )
                a.update({'height': out_image.shape[1], 'width': out_image.shape[2], 'crs':'epsg:26910', 'affine':aff, 'transform':trans })     
                new_pic_path = os.path.join(path,pic['name'].replace('.tif','') +'_'+ str(intersect_fields.iloc[i]["OBJECTID"])+'.tif')
                with rasterio.open(new_pic_path, 'w', **a) as dst:
                    dst.write(out_image)
                dst.close()

In [43]:
path = 'D:\\NAIP GEOTIFF'
pic_path = os.path.join(os.path.join(path, pics.iloc[0]['name'].replace('.tif', '')),pics.iloc[0]['name'] )
for i in range(len(intersect_fields)):
    new_geoms = [mapping(intersect_fields.iloc[i].geometry)]
    with rasterio.open(pic_path) as src:
        out_image, out_transform = mask(src, new_geoms, crop = True)
        if(out_image.shape[1]*out_image.shape[2]*0.6**2 > .5*intersect_fields.iloc[i]["geometry"].area):
            a = src.profile
            aff = Affine(src.affine[0], src.affine[1], intersect_fields.iloc[i]['geometry'].bounds[0], src.affine[3], src.affine[4], geoms[0].bounds[3])
            trans = (intersect_fields.iloc[i]['geometry'].bounds[0], src.transform[1], src.transform[2],intersect_fields.iloc[i]['geometry'].bounds[3], src.transform[4], src.transform[5] )
            a.update({'height': out_image.shape[1], 'width': out_image.shape[2], 'crs':'epsg:26910', 'affine':aff, 'transform':trans })     
            new_pic_path = os.path.join(path,pics.iloc[0]['name'].replace('.tif','') +'_'+ str(intersect_fields.iloc[i]["OBJECTID"])+'.tif')
            with rasterio.open(new_pic_path, 'w', **a) as dst:
                dst.write(out_image)
            dst.close()

C:\Users\marce\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\marce\Anaconda3\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [44]:
for pic in pics:
    print(pics)

        date                            name  \
0   20160711  m_3812101_se_10_h_20160711.tif   
1   20160711  m_3812101_sw_10_h_20160711.tif   
2   20160711  m_3812102_se_10_h_20160711.tif   
3   20140726  m_3812102_sw_10_1_20140726.tif   
4   20160711  m_3812102_sw_10_h_20160711.tif   
5   20160711  m_3812109_ne_10_h_20160711.tif   
6   20160711  m_3812109_nw_10_h_20160711.tif   
7   20160711  m_3812109_se_10_h_20160711.tif   
8   20160711  m_3812109_sw_10_h_20160711.tif   
9   20160711  m_3812110_ne_10_h_20160711.tif   
10  20160711  m_3812110_nw_10_h_20160711.tif   
11  20160711  m_3812110_se_10_h_20160711.tif   
12  20160711  m_3812110_sw_10_h_20160711.tif   
13  20160710  m_3812111_nw_10_h_20160710.tif   
14  20160710  m_3812111_se_10_h_20160710.tif   
15  20160710  m_3812111_sw_10_h_20160710.tif   
16  20140726  m_3812117_ne_10_1_20140726.tif   
17  20160711  m_3812117_ne_10_h_20160711.tif   
18  20140726  m_3812117_nw_10_1_20140726.tif   
19  20160711  m_3812117_nw_10_h_20160711